In [1]:
import pandas as pd


data = pd.read_csv(
    "/Users/roysung/py_projects/hm_personal_dataset/transactions_train.csv",
    dtype={'customer_id': 'category', 'article_id': 'category', 'sales_channel_id': 'category'},
    parse_dates=['t_dat']). \
    assign(week=lambda x: (x['t_dat'].max() - x['t_dat']).dt.days // 7)

user_dt = data. \
    groupby(['customer_id', 't_dat'], observed=True). \
    agg(item_num=('article_id', 'size')). \
    reset_index(). \
    sort_values(['customer_id', 't_dat']). \
    assign(duration=lambda x: x.groupby('customer_id')['t_dat'].diff().dt.days)

## 購買日期 2018-09-20 ~ 2020-09-22
## 平均回購天數 48 天，中位數 22 天
## 每次平均購買件數 3.5 件商品，中位數 2 件
user_dt.describe()

## 用戶的平均回購天數 90 天，中位數 60 天
## 用戶平均購買 6.66 次，中位數 3 次
## 用戶每次的平均購買數量 3.5 件商品，中位數 2.83 件
customer_info = user_dt. \
    groupby('customer_id', observed=True). \
    agg(
        buy_num=('t_dat', 'count'),
        avg_interval=('duration', 'mean'),
        avg_item_num=('item_num', 'mean'))
customer_info.describe()

# CF 模型去除雜訊
## 只買過 1 次且商品只有 1 件的人: 131,514
customer_info[(customer_info['buy_num']==1) & (customer_info['avg_item_num']==1)]


In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn.models import LightGCN
from torch_geometric.utils import degree
from torch_geometric.utils.sparse import to_torch_coo_tensor


TRANSACTIONS_DIR = "/Users/roysung/py_projects/hm_personal_dataset/transactions_train.csv"
CUSTOMERS_DIR = "/Users/roysung/py_projects/hm_personal_dataset/customers.csv"
ARTICLES_DIR = "/Users/roysung/py_projects/hm_personal_dataset/articles.csv"


class SequenceEncoder:
    """
    :param model_name: 预训练编码模型的名称
    :param device: 选择将数据载入至cuda或者cpu
    """
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()


class OneHotEncoder(object):
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        # 对题材集合进行编码
        df = df.astype('str').astype('category')
        mapping = {cls: i for i, cls in enumerate(df.unique())}
 
        # 采用类似one-hot编码，编码长度为num_genres，输出维度为[num_samples, num_genres]
        x = torch.zeros(len(df), len(mapping))
        for i, cls in enumerate(df.values):
            # 同一个电影存在多个题材，在归属于该题材的编码位上置1，其他位置0
            x[i, mapping[cls]] = 1
        return x


class AgeEncoder(OneHotEncoder):
    def __init__(self, bin):
        self.bin = bin
        self.labels = range(len(bin) - 1)

    def __call__(self, df):
        df = pd.cut(df, bins=self.bin, labels=self.labels)
        return super().__call__(df)


def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, dtype={index_col: 'category'}, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping


def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, p_test=0.4, **kwargs):
    df = pd.read_csv(
        path,
        dtype={src_index_col: 'category', dst_index_col: 'category'},
        parse_dates=['t_dat']). \
        assign(week=lambda x: (x['t_dat'].max() - x['t_dat']).dt.days // 7)

    # filter
    low_freq_user = df. \
        groupby(['customer_id', 't_dat'], observed=True). \
        agg(item_num=('article_id', 'size')). \
        reset_index(). \
        groupby('customer_id', observed=True). \
        agg(
            buy_num=('t_dat', 'count'),
            avg_item_num=('item_num', 'mean')). \
        query('buy_num == 1 and avg_item_num == 1').index
    
    df = df[~df['customer_id'].isin(low_freq_user)]. \
        sort_values([src_index_col, dst_index_col])

    # create edge index
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    # create the mask for test data
    test_mask = torch.tensor((df['week'] == 0).to_numpy())

    # create edge attributes
    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr, test_mask


# c_x, c_mapping = load_node_csv(
#     CUSTOMERS_DIR,
#     index_col="customer_id",
#     encoders={
#         "FN": OneHotEncoder(),
#         "Active": OneHotEncoder(),
#         "club_member_status": OneHotEncoder(),
#         "fashion_news_frequency": OneHotEncoder(),
#         "age": AgeEncoder([0, 30, 40, 50, 60, 100]) })
_, c_mapping = load_node_csv(CUSTOMERS_DIR, "customer_id")
_, a_mapping = load_node_csv(ARTICLES_DIR, "article_id")
edge_index, _, test_mask = load_edge_csv(
    TRANSACTIONS_DIR,
    src_index_col='customer_id',
    src_mapping=c_mapping,
    dst_index_col='article_id',
    dst_mapping=a_mapping)

data = HeteroData({
    "customer": {"num_nodes": len(c_mapping)}, #{"x": c_x},
    "article": {"num_nodes": len(a_mapping)},
    ("customer", "rates", "article"): {"edge_index": edge_index}
})

/Users/roysung/miniforge3/envs/torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cpu"#torch.device("mps")

num_users, num_items = len(c_mapping), len(a_mapping)
data = data.to_homogeneous().to(device)

# Use all message passing edges as training labels:
batch_size = 8192
train_edge_index = data.edge_index[:, ~test_mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_index.size(1)),
    shuffle=True,
    batch_size=batch_size)

test_edge_index = data.edge_index[:, test_mask]

# Build model
model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [3]:
def _dcg(target: torch.Tensor) -> torch.Tensor:
    denom = torch.log2(torch.arange(target.shape[-1], device=target.device) + 2.0)
    return (target / denom).sum(dim=-1)


def train_step(index: torch.Tensor, num_users: int, num_items: int, optimizer):
    # Sample positive and negative labels.
    pos_edge_index = train_edge_index[:, index]
    neg_edge_index = torch.stack([
        pos_edge_index[0],
        torch.randint(
            low=num_users,
            high=num_users + num_items,
            size=(index.numel(), ),
            device=device)
    ], dim=0)
    edge_label_index = torch.cat(
        [pos_edge_index, neg_edge_index], dim=1)

    optimizer.zero_grad()
    pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

    loss = model.recommendation_loss(
        pos_rank, neg_rank, node_id=edge_label_index.unique())
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test_step(test_mask: torch.Tensor, k: int, num_users: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, item_emb = emb[:num_users], emb[num_users:]

    ndcg = precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ item_emb.t()

        # Exclude training edges:
        mask = ((train_edge_index[0] >= start) &
                (train_edge_index[0] < end))
        logits[train_edge_index[0, mask] - start,
               train_edge_index[1, mask] - num_users] = float('-inf')

        # Computing ndcg, precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_index[0] >= start) &
                (data.edge_index[0] < end) &
                test_mask)
        ground_truth[data.edge_index[0, mask] - start,
                     data.edge_index[1, mask] - num_users] = True
        node_count = degree(
            data.edge_index[0, mask] - start,
            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        sorted_truth = ground_truth.gather(1, topk_index)
        ideal_truth = torch.sort(ground_truth, descending=True).values[:, :20]

        sorted_dcg = _dcg(sorted_truth)
        ideal_dcg = _dcg(ideal_truth)

        ndcg += float((sorted_dcg[ideal_dcg != 0] / ideal_dcg[ideal_dcg != 0]).sum())
        precision += float((sorted_truth.sum(dim=-1) / k).sum())
        recall += float((sorted_truth.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())
    return ndcg / total_examples, precision / total_examples, recall / total_examples

In [5]:
for epoch in range(1, 2):
    step = 1
    for index in tqdm(train_loader):
        loss = train_step(index, num_users, num_items, optimizer)
        # if step % 50 == 0:
        #     ndcg, precision, recall = test_step(test_mask, 20, num_users)
        #     print(f'Step: {step:05d}, Loss: {loss:.4f}, NDCG@20: {ndcg:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}')
        # step += 1
    ndcg, precision, recall = test_step(test_mask, 20, num_users)
    print(f'Step: {step:05d}, NDCG@20: {ndcg:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}')

  1%|▏         | 50/3836 [13:16<79:52:39, 75.95s/it]

Step: 00050, Loss: 0.6931, NDCG@20: 0.0007, Precision@20: 0.0002, Recall@20: 0.0013


  3%|▎         | 100/3836 [26:43<79:45:02, 76.85s/it]

Step: 00100, Loss: 0.6931, NDCG@20: 0.0007, Precision@20: 0.0002, Recall@20: 0.0013


  4%|▍         | 150/3836 [40:08<78:24:22, 76.58s/it]

Step: 00150, Loss: 0.6930, NDCG@20: 0.0011, Precision@20: 0.0003, Recall@20: 0.0018


  5%|▌         | 199/3836 [51:38<15:43:54, 15.57s/it]


KeyboardInterrupt: 

In [4]:
for epoch in range(1, 2):
    step = 1
    for index in tqdm(train_loader):
        loss = train_step(index, num_users, num_items, optimizer)
        if step % 50 == 0:
            ndcg, precision, recall = test_step(test_mask, 20, num_users)
            print(f'Step: {step:05d}, Loss: {loss:.4f}, NDCG@20: {ndcg:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}')
        step += 1

  2%|▏         | 50/2641 [13:23<55:38:51, 77.32s/it]

Step: 00050, Loss: 0.6931, NDCG@20: 0.0024, Precision@20: 0.0019, Recall@20: 0.0018


  4%|▍         | 100/2641 [26:43<54:35:15, 77.34s/it]

Step: 00100, Loss: 0.6931, NDCG@20: 0.0023, Precision@20: 0.0017, Recall@20: 0.0016


  6%|▌         | 150/2641 [40:01<53:30:20, 77.33s/it]

...

Step: 01500, Loss: 0.3204, NDCG@20: 0.0138, Precision@20: 0.0105, Recall@20: 0.0109


 59%|█████▊    | 1550/2641 [6:46:47<21:38:59, 71.44s/it]

Step: 01550, Loss: 0.3175, NDCG@20: 0.0137, Precision@20: 0.0104, Recall@20: 0.0108


 61%|██████    | 1599/2641 [6:55:45<3:10:03, 10.94s/it] 

: 